In [1]:
#This table is necessary in order to convert state name to abbreviation quickly
#https://github.com/nquandt98/project1cosc3570nquandt


us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'District of Columbia' : 'MD',
}


In [35]:
from datascience import *
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import numpy
import urllib
import requests

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/nquandt98/project1cosc3570nquandt/master/cc_institution_details.csv', 
                 encoding='ANSI')

#add state abbreviations to table
stateindex = df.columns.get_loc("state")+1
i=0
result_array = numpy.array([])
for row in df.itertuples():
    result_array = numpy.append(result_array, str(us_state_abbrev[row[stateindex]]))
    i+=1
    print(i,end="\r")
df['state_code'] = pd.Series(result_array)

#create a smaller table to change up as needed
t = df[['chronname','control','FIPS','retain_value','awards_per_value','aid_value','aid_percentile','exp_award_value','state','state_code','level']]
t = t.sort_values(by=['retain_value'],ascending = False)

#eliminate 0 rows
t = t[t.exp_award_value != 0]
t = t[t.aid_value != 0]
t.loc[t.exp_award_value == 0, 'exp_award_value'] = numpy.nan
t.dropna(inplace=True)

#create ratio value
t['aid_to_exp'] = t['aid_value']/t['exp_award_value']

#select only public and 4year
t = t[t.control == 'Public']
t = t[t.level == '4-year']

#copy table for cleaned graph
t2 = t
i='aid_to_exp'
t2.loc[t2.aid_to_exp == 0, 'aid_to_exp'] = numpy.nan
t2.dropna(inplace=True)
t2['Log_' + i] = numpy.abs(numpy.log(t2[i]))
q75, q25 = numpy.percentile(t2.Log_aid_to_exp.dropna(), [75 ,25])
iqr = q75 - q25

#use IQR to remove outliers statistically
min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5)
t2['Outlier'] = 0
i ='Log_aid_to_exp'
t2.loc[t2[i] < min, 'Outlier'] = 1
t2.loc[t2[i] > max, 'Outlier'] = 1
t2 = t2[t2.Outlier == 0]

#create a trace for the graphs
barGraphOriginalTrace = go.Bar(y=t.aid_to_exp,x=t.retain_value, name= 'Original')
barGraphCleanedTrace = go.Bar(y=t2.aid_to_exp,x=t2.retain_value, name = 'Outliers Removed')
scatterGraphOriginalTrace = go.Scatter(
    x=t.retain_value,y=t.aid_to_exp,
    mode = 'markers',
    name = 'Original'
)
scatterGraphCleanedTrace = go.Scatter(
    x=t2.retain_value,y=t2.aid_to_exp,
    mode = 'markers',
    name = 'Outliers Removed'
)

#compile traces over eachother
bardata = [barGraphOriginalTrace,barGraphCleanedTrace]
scatterdata = [scatterGraphOriginalTrace,scatterGraphCleanedTrace]

#create layouts
layout = {'xaxis': {'title': 'Retention Percentage of Students 1st-2nd Year'},
  'yaxis': {'title': 'Ratio of Aid Given to Cost of Attending'},
  'barmode': 'group',
  'title': 'Comparison'}
layout2 = {'xaxis': {'title': 'Retention Percentage of Students 1st-2nd Year'},
  'yaxis': {'title': 'Ratio of Aid Given to Cost of Attending'},
    'title': 'Scatter Comparison'}
barFigure = go.Figure(data = bardata,layout= layout)
ScatterFigure = go.Figure(data = scatterdata, layout = layout2)

colorscale1 = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]

histogramFigure = ff.create_2d_density(t2.aid_to_exp,
    t2.retain_value, colorscale=colorscale1,
    hist_color='rgb(0, 68, 124)', point_size=3
)

#plot graphs to plotly
py.iplot(histogramFigure, filename='histogram_subplots')
py.iplot(ScatterFigure, filename = 'ScatterPlot')
py.iplot(barFigure, filename = 'BarChart')




In [3]:
#use cleaned data for futher research
#create choropleth for states ratio values
values = t2['aid_to_exp'].values
print(values.max())
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = t2['state_code'],
        z = values.astype(float),
        locationmode = 'USA-states',
        text = "",
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Ratio of Aid Given to Cost of Attending"
        )
    ) ]
layout = dict(
        title = 'Aid of Schools',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='d3-cloropleth-map1')

0.2359628600672613


In [4]:
#use cleaned data for futher research

#create choropleth for states retention values
values2 = t2['retain_value'].values
print(values2.max())
scl2 = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]


data2 = [ dict(
        type='choropleth',
        colorscale = scl2,
        autocolorscale = True,
        locations = t2['state_code'],
        z = values2.astype(float),
        locationmode = 'USA-states',
        text = "",
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Percent of Retainment"
        )
    ) ]
layout2 = dict(
        title = 'Retention Rate',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

fig2 = dict(data=data2, layout=layout2)

py.iplot(fig2, filename='d3-cloropleth-map2')

100.0


In [27]:
#create a sample table for the paper
table1 = Table.read_table('https://raw.githubusercontent.com/nquandt98/project1cosc3570nquandt/master/cc_institution_details.csv', encoding='ANSI')

table1 = table1.sample(10).select('chronname','state','control','student_count','aid_value','exp_award_value')
table1 = table1.with_column('aid_to_exp', table1.column('aid_value')/table1.column('exp_award_value'))
tabledf = table1.to_df()
tablefile = go.Table(
    header=dict(values=['College','State','Type','Student Count','Financial Aid Given', 'Cost of Attending til Gradution', 'Aid/Cost'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[tabledf.chronname,tabledf.state,tabledf.control,tabledf.student_count,tabledf.aid_value,tabledf.exp_award_value,tabledf.aid_to_exp],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

datatable = [tablefile] 
figtable = go.Figure(data=datatable)
py.iplot(datatable, filename = 'example_sample')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning:

divide by zero encountered in true_divide



In [34]:
#create a scatter of california schools only
#attempt to say california has higher rates and better ratios
californiaschools = t[t.state == 'California']

caltrace = go.Scatter(
    x=californiaschools.retain_value,y=californiaschools.aid_to_exp,
    mode = 'markers',
    name = 'California Only',
    marker = dict(
        size = 10,
        color = 'rgba(152, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)
linetrace = go.Scatter(
    x = [70,100],
    y = [.15,.15],
    mode = 'lines',
    name = '.15 Line')

calscatterdata = [caltrace,linetrace]
callayout = {'xaxis': {'title': 'Retention Percentage of Students 1st-2nd Year'},
  'yaxis': {'title': 'Ratio of Aid Given to Cost of Attending'},
    'title': 'Scatter California'}
calfig = go.Figure(data = calscatterdata, layout = callayout)


py.iplot(calfig, filename = 'CalScatterPlot')


In [ ]:
#this is extra code that I had use to convert lat long to FIPS,
#but because it is a json response parser the code takes to long to run
#everytime thus I just replaced my entire csv with the new one

#latindex = df.columns.get_loc("lat_y")+1
#lonindex = df.columns.get_loc("long_x")+1
#i=0
#result_array = numpy.array([])
#for row in df.itertuples():
    #url = ('https://geo.fcc.gov/api/census/block/find?latitude=' + str("%.2f" % float(row[latindex])) + '&longitude=' + str("%.2f" % float(row[lonindex])) + '&format=json')
    #response = requests.get(url)
    #locdata = response.json()
   # result_array = numpy.append(result_array, str(locdata['County']['FIPS']))
  #  i+=1
 #   print(i,end="\r")
#df['FIPS'] = pd.Series(result_array)

#df.to_csv('df21.csv', encoding='ANSI', index=False)
